In [6]:
!pip install pycaret[full]


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install optuna 


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import mlflow
from sklearn.metrics import roc_auc_score,balanced_accuracy_score

In [9]:
root = Path('/').resolve()
X_directory = Path('Attrition_Application\/artifacts\data_transformed_artifact\X_train_trfm.csv')
y_directory = Path('Attrition_Application\/artifacts\data_transformed_artifact\Y_train_trfm.csv')
X = Path.joinpath(root,X_directory)
Y = Path.joinpath(root,y_directory)

In [10]:
X = pd.read_csv(X,header=None)
y = pd.read_csv(Y,header=None)

In [11]:
data = X
data['target'] = y[1]

In [12]:
X.columns = [str(feature) for feature in X.columns]

In [13]:
y = y[1]

In [14]:
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

In [15]:
# init setup on exp
exp.setup(data, target = 'target', session_id = 123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(1967, 26)"
4,Transformed data shape,"(1967, 26)"
5,Transformed train set shape,"(1376, 26)"
6,Transformed test set shape,"(591, 26)"
7,Numeric features,25
8,Rows with missing values,0.1%
9,Preprocess,True


In [16]:
exp.models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [17]:
best_models = exp.compare_models(include=['lr','dt','rf','gbc'],cross_validation=True,fold=5,n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9164,0.9588,0.8619,0.9680,0.9113,0.8328,0.8385,0.1680
gbc,Gradient Boosting Classifier,0.9157,0.9550,0.8575,0.9703,0.9101,0.8313,0.8374,0.2260
lr,Logistic Regression,0.8641,0.9340,0.8488,0.8761,0.8619,0.7282,0.7290,2.6700
dt,Decision Tree Classifier,0.8597,0.8597,0.8749,0.8490,0.8616,0.7194,0.7201,1.8400


In [18]:
best_models

[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='sqrt',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_samples_leaf=1,
                        min_samples_split=2, min_weight_fraction_leaf=0.0,
                        monotonic_cst=None, n_estimators=100, n_jobs=-1,
                        oob_score=False, random_state=123, verbose=0,
                        warm_start=False),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='log_loss', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_samples_leaf=1,
                            min_samples_split=2, min_weight_fraction_leaf=0.0,
                            n_estimators=100, n_iter_no_change=None,
    

In [19]:
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [20]:
models_dict = {
               'LogisticRegression': LogisticRegression,
               'RandomForestClassifier': RandomForestClassifier, 
               'DecisionTreeClassifier': DecisionTreeClassifier,
               'SGDClassifier':SGDClassifier,
               'GradientBoostingClassifier':GradientBoostingClassifier
               }



top_models = [models_dict[model.__class__.__name__] for model in best_models]

In [21]:
top_models

[sklearn.ensemble._forest.RandomForestClassifier,
 sklearn.ensemble._gb.GradientBoostingClassifier,
 sklearn.linear_model._logistic.LogisticRegression]

In [22]:
def LogisticRegObj(trial):
  lr_C = trial.suggest_uniform('C', 0.0, 1.0)

  classifier_obj = LogisticRegression(C=lr_C)
  classifier_obj = classifier_obj.fit(X,y)
  pred = classifier_obj.predict(X)
  score = roc_auc_score(y,pred)

  return score

def RandomForObj(trial):
  rfc_n_estimators = trial.suggest_int('n_estimators', 50, 250, 25)
  rfc_criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
  rfc_min_samples_split  = trial.suggest_uniform('min_samples_split',0.2, 1.0)
  rfc_max_depth = trial.suggest_int('max_depth', 2, 10, 2)

  classifier_obj = RandomForestClassifier(
                                   n_estimators=rfc_n_estimators,
                                   criterion=rfc_criterion,
                                   min_samples_split = rfc_min_samples_split,
                                   max_depth = rfc_max_depth
                                   )
  classifier_obj = classifier_obj.fit(X,y)
  
  pred = classifier_obj.predict(X)
  score = roc_auc_score(y,pred)

  return score

def SGDRegObj(trial):
    sgd_eta0 = trial.suggest_uniform('eta0',0.1, 1.0)

    sgd_loss = trial.suggest_categorical('loss', ['hinge',
                                                      'log_loss',
                                                      'modified_huber',
                                                      'squared_hinge',
                                                      'perceptron',
                                                      'squared_error',
                                                      'huber'])
    sgd_learning_rate = trial.suggest_categorical('learning_rate', ['constant',
                                                      'optimal',
                                                      'invscaling',
                                                      'adaptive'])

    classifier_obj = SGDClassifier(loss=sgd_loss,
                                       eta0 = sgd_eta0,
                                       learning_rate=sgd_learning_rate)

    classifier_obj = classifier_obj.fit(X,y)
    pred = classifier_obj.predict(X)
    score = roc_auc_score(y,pred)

    return score

def DecisionTreeClassObj(trial):
    criterion = trial.suggest_categorical("criterion", ["log_loss", "gini",'entropy'])
    max_depth = trial.suggest_int(
          "n_estimators", 3, 8, 1
      )
    
    subsample = trial.suggest_float(
          "subsample", 0.3, 0.9, log=True
      )
    
    min_samples_leaf = trial.suggest_float(
        "min_samples_leaf", 0.3, 0.9, log=True
    )

    min_samples_split = trial.suggest_float(
          "min_samples_split", 0.3, 0.9, log=True
    )
    
    classifier_obj = DecisionTreeClassifier(
                                        criterion = criterion,
                                        max_depth=max_depth,  
                                        min_samples_split = min_samples_split,
                                        subsample=subsample,
                                        min_samples_leaf=min_samples_leaf
                                        )

    classifier_obj = classifier_obj.fit(X,y)
    pred = classifier_obj.predict(X)
    score = roc_auc_score(y,pred)

    return score


def GradientBoostClassObj(trial):
  loss = trial.suggest_categorical("loss", ["log_loss", "exponential"])
  max_depth = trial.suggest_int(
          "n_estimators", 3, 8, 1
      )
  learning_rate = trial.suggest_float(
        "learning_rate", 0.0005, 0.05, log=True
    )
  n_estimators = trial.suggest_int(
        "n_estimators", 50, 350, 50
    )
   
  subsample = trial.suggest_float(
        "subsample", 0.3, 0.9, log=True
    )
   
  min_samples_leaf = trial.suggest_float(
        "min_samples_leaf", 0.3, 0.9, log=True
  )

  min_samples_split = trial.suggest_float(
        "min_samples_split", 0.3, 0.9, log=True
  )
   
  classifier_obj = GradientBoostingClassifier(
                                       n_estimators = n_estimators,
                                       max_depth=max_depth,
                                       loss=loss,
                                       learning_rate = learning_rate,
                                       subsample=subsample,
                                       min_samples_split=min_samples_split,
                                       min_samples_leaf=min_samples_leaf
                                       )

  classifier_obj = classifier_obj.fit(X,y)
  pred = classifier_obj.predict(X)
  score = roc_auc_score(y,pred)

  return score

objectives = {
    'LogisticRegression':LogisticRegObj,
    'RandomForestClassifier': RandomForObj,
    'SGDClassifier': SGDRegObj,
    'GradientBoostingClassifier':GradientBoostClassObj,
    'DecisionTreeClassifier': DecisionTreeClassObj
}

In [23]:
X.fillna(0,inplace=True)

In [27]:
mlflow.set_tracking_uri(uri='http://127.0.0.1:1500')
best_models = {}

def evaluate(actual,prediction):
  return {
          'roc_score':roc_auc_score(actual,prediction),
          'balance_accuracy_score':balanced_accuracy_score(actual,prediction)
          }


for index,model in enumerate([model for model in top_models]):
  model_name = model().__class__.__name__
  experiment = mlflow.set_experiment(experiment_name=f'experiement_{model_name}')

  study = optuna.create_study(direction="maximize")
  study.optimize(objectives[model_name], n_trials=200)

  params = study.best_params
  best_models[model_name] =  {'score':study.best_value,
                                              'params':study.best_params,
                                              'model':None}
  params_list = list(study.best_params.keys())

  print(params_list,model_name)

  plot_contour_fig = optuna.visualization.plot_contour(study, params=params_list)
  plot_parallel_coordinate_fig = optuna.visualization.plot_parallel_coordinate(study, params=params_list)
  plot_optimization_history_fig = optuna.visualization.plot_optimization_history(study)

  with mlflow.start_run(experiment_id=experiment.experiment_id):

    model = model(**params)
    model.fit(X,y)
    best_models[model_name]['model'] = model
    predict = model.predict(X)

    mlflow.log_param("models score", study.best_value)
    mlflow.log_metrics(evaluate(y,predict))
    mlflow.log_params(model.get_params())
    mlflow.log_figure(plot_contour_fig, "plot_contour_fig.png")
    mlflow.log_figure(plot_parallel_coordinate_fig, "plot_parallel_coordinate_fig.png")
    mlflow.log_figure(plot_optimization_history_fig, "plot_optimization_history_fig.png")


[I 2025-03-29 17:38:46,043] A new study created in memory with name: no-name-51e44249-3c9a-4f4e-b53e-51d2121f64ab


[I 2025-03-29 17:38:46,202] Trial 0 finished with value: 0.9923702950152594 and parameters: {'n_estimators': 75, 'criterion': 'log_loss', 'min_samples_split': 0.4566050979741286, 'max_depth': 8}. Best is trial 0 with value: 0.9923702950152594.
[I 2025-03-29 17:38:46,602] Trial 1 finished with value: 1.0 and parameters: {'n_estimators': 225, 'criterion': 'entropy', 'min_samples_split': 0.3694658609314691, 'max_depth': 4}. Best is trial 1 with value: 1.0.
[I 2025-03-29 17:38:46,900] Trial 2 finished with value: 0.9847405900305188 and parameters: {'n_estimators': 175, 'criterion': 'log_loss', 'min_samples_split': 0.5556842733693363, 'max_depth': 4}. Best is trial 1 with value: 1.0.
[I 2025-03-29 17:38:46,994] Trial 3 finished with value: 1.0 and parameters: {'n_estimators': 50, 'criterion': 'gini', 'min_samples_split': 0.229954079522755, 'max_depth': 10}. Best is trial 1 with value: 1.0.
[I 2025-03-29 17:38:47,148] Trial 4 finished with value: 0.5 and parameters: {'n_estimators': 100, 'cr

['n_estimators', 'criterion', 'min_samples_split', 'max_depth'] RandomForestClassifier


[I 2025-03-29 17:40:04,084] A new study created in memory with name: no-name-6a817e2e-8af4-4012-84de-dd5632adc1db
[I 2025-03-29 17:40:04,104] Trial 0 finished with value: 0.5 and parameters: {'loss': 'exponential', 'n_estimators': 7, 'learning_rate': 0.016558024024563484, 'subsample': 0.8589458873864946, 'min_samples_leaf': 0.6177183081856287, 'min_samples_split': 0.5093791289947401}. Best is trial 0 with value: 0.5.
[I 2025-03-29 17:40:04,123] Trial 1 finished with value: 0.5 and parameters: {'loss': 'log_loss', 'n_estimators': 3, 'learning_rate': 0.04193546674704217, 'subsample': 0.41918971026185925, 'min_samples_leaf': 0.31442363034499127, 'min_samples_split': 0.7250552205882812}. Best is trial 0 with value: 0.5.
[I 2025-03-29 17:40:04,142] Trial 2 finished with value: 0.5 and parameters: {'loss': 'exponential', 'n_estimators': 7, 'learning_rate': 0.0016992911670141529, 'subsample': 0.32371664900191505, 'min_samples_leaf': 0.42904543369199755, 'min_samples_split': 0.6657890403246457

🏃 View run learned-bee-80 at: http://127.0.0.1:1500/#/experiments/1/runs/77793f7212be4d79af179a4b9ec9dae4
🧪 View experiment at: http://127.0.0.1:1500/#/experiments/1


...

['loss', 'n_estimators', 'learning_rate', 'subsample', 'min_samples_leaf', 'min_samples_split'] GradientBoostingClassifier


[I 2025-03-29 17:40:57,251] A new study created in memory with name: no-name-3152180d-c593-4627-8a74-c602e7711fec
[I 2025-03-29 17:40:57,336] Trial 0 finished with value: 1.0 and parameters: {'C': 0.5816497547216986}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,415] Trial 1 finished with value: 1.0 and parameters: {'C': 0.7407601348228648}. Best is trial 0 with value: 1.0.


🏃 View run intrigued-shad-651 at: http://127.0.0.1:1500/#/experiments/2/runs/af1104e55151482a91931574327931ed
🧪 View experiment at: http://127.0.0.1:1500/#/experiments/2


[I 2025-03-29 17:40:57,503] Trial 2 finished with value: 1.0 and parameters: {'C': 0.11406310954455567}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,644] Trial 3 finished with value: 1.0 and parameters: {'C': 0.07001285638120958}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,724] Trial 4 finished with value: 1.0 and parameters: {'C': 0.5591267724412591}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,805] Trial 5 finished with value: 1.0 and parameters: {'C': 0.9515725763471286}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,885] Trial 6 finished with value: 1.0 and parameters: {'C': 0.18985488971148212}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:57,996] Trial 7 finished with value: 1.0 and parameters: {'C': 0.9461463316820526}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:58,077] Trial 8 finished with value: 1.0 and parameters: {'C': 0.258927842056356}. Best is trial 0 with value: 1.0.
[I 2025-03-29 17:40:58,172] Trial 9 fin

['C'] LogisticRegression
🏃 View run bald-snake-333 at: http://127.0.0.1:1500/#/experiments/3/runs/d4c09366da384380bce89ea66ebadf17
🧪 View experiment at: http://127.0.0.1:1500/#/experiments/3


In [38]:
models = sorted(best_models.items(),key=lambda item:item[1]['score'],reverse=True)
best_model = [model[-1]['model'] for model in models][0]

In [39]:
best_model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=4, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=0.3694658609314691,
                       min_weight_fraction_leaf=0.0, monotonic_cst=None,
                       n_estimators=225, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)